                                    POWER PLANTS DATA PROCESSING

Each part of the following script was used to proccess the raw data for power plants units of the Dispa-SET_Unleash project. Read instructions carefully to follow all the process until final results stept by step.

            1. ZONE AND FOLDER CREATION

1.1. Run the following two cell to introduce the name of the folder where all the data is goint to be downloaded.

    For European country names use the ISO 3166-1 standars i.e. AT, BE, BG, CH.... etc. to give the zone folder name
    For non European countries it would rather to call the zone folder name with the same word of how it is defined in the data to be downloaded and processed. e.g. If it is downloading a csv file with all power plants of Spain but just the units fo Pamplona city are wanted, and in the donwloaded file, Pamplona is refering with the acronim PMPLN; set the zone_file_name variable with the word PMPLN.

1.2. For the folder path is recommended keep the Dispa-SET directory structure:

    /dispa-SET/home/directory/path/RawData/PowerPlants
    e.g.
    /home/ray/Dispa-SET_Unleash/RawData/PowerPlants

In [1]:
# Define variables
zone_folder_name = "DE"  # Replace with the desired folder name
zone_folder_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants"  # Replace with the desired parent folder path

# Call the external Python script with defined variables
#%run -i Zone_Folder_Creation_1.py {zone_folder_name} {zone_folder_path}

In [2]:
import os
#import sys

def create_folder(folder_name, parent_folder_path):
    """
    Creates a folder with the specified name in the specified parent folder.
    
    Parameters:
    - folder_name: The name of the folder to be created (string)
    - parent_folder_path: The path of the parent folder (string)
    
    Returns:
    - folder_path: The path of the folder created (string)
    """
    folder_path = os.path.join(parent_folder_path, folder_name)
    os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist
    return folder_path

# Input variables from command line arguments
#zone_folder_name = sys.argv[1]
#zone_folder_path = sys.argv[2]

# Call create_folder function with provided arguments
power_plants_raw_data_folder_path = create_folder(zone_folder_name, zone_folder_path)

# Print the path of the folder created
print(f"Folder {zone_folder_name} created at: {zone_folder_path}")


# Print the values of the input variables
print("zone_folder_name:", zone_folder_name)
print("zone_folder_path:", zone_folder_path)   
print("power_plants_raw_data_folder_path:", power_plants_raw_data_folder_path) 

Folder DE created at: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [3]:
print (f"zone_folder_name: {zone_folder_name}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\nzone_folder_path: {zone_folder_path}")

zone_folder_name: DE
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants


            2. POWER PLANTS RAW DATA DOWNLOAD

2.1. Run the following two cells to download files containers of raw data related power plants units of the zone previously created i.e. BE, UK, CH,... etc.

2.2. To keep the Dispa-SET directories structure and file names, is recommended to give the year which the data is referring to as the downloaded file name. Alsom there may be different power plant data sources for an only country, zone, region, etc; so it is also recommended to add a numeric suffix to the file name to difference between files. Aditionally, take account to give the appropiate file extension to the downloaded file name, .csv in this case:

    power_plants_raw_data_file_name = "20##-##.csv"
    e.g.
    power_plants_raw_data_file_name = "2020-01.csv"

2.3. All the scripts related to this MAIN code are desinged to proccess the original files from the following web sources at 2020:

    For
        BE (Belgium)
            conventional and renewable power plants 
                https://opendata.elia.be/pages/home/
    For
        DE (Germany)
        DK (Denmark)
        FR (France)
        UK (United Kingdom)
        PL (Poland)
        CH (Switzerland)
        CZ (Czech Republic)
        SE (Sweden)
            renewable power plants only
                https://data.open-power-system-data.org/renewable_power_plants/2020-08-25
    For
        DE (Germany)
        AT (Austria)
        CH (Switzerland)
        CZ (Czech Republic)
        DK (Denmark)
        ES (Spain)
        FI (Finland)
        FR (France)
        IT (Italy)
        NL (the Netherlands)
        NO (Norway)
        PL (Poland)
        SE (Sweden)
        SK (Slovakia)
        SI (Slovenia)
        UK (United Kingdom)
            conventional power plants only
                https://data.open-power-system-data.org/conventional_power_plants/2020-10-01

In [4]:
# Input file name and download link from user
power_plants_raw_data_file_name = "2020-01.csv"
power_plants_raw_data_download_link = "https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv"

# Call the download_script.py script using %run
#%run -i Power_Plants_Data_Download_1.py {power_plants_raw_data_download_link} {power_plants_raw_data_folder_path} {power_plants_raw_data_file_name}

In [5]:
import os
import csv
import requests

def download_file(url, folder_path, file_name):
    """
    Downloads a file from the given URL and saves it to the specified folder path with the given filename and extension.
    
    Parameters:
    - url: The URL to download the file from (string)
    - folder_path: The path of the folder where the file will be saved (string)
    - file_name: The name of the file (string)
    
    Returns:
    - downloaded_file_path: The full path of the downloaded file (string)
    """
    downloaded_file_path = os.path.join(folder_path, file_name)
    response = requests.get(url)
    if response.status_code == 200:
        with open(downloaded_file_path, 'wb') as f:
            f.write(response.content)
        return downloaded_file_path
    else:
        print(f"Error downloading file: {response.status_code}")
        return None

# Input variables from command line arguments
#power_plants_file_name = sys.argv[1]
#download_link = sys.argv[2]
#zone_folder_name = sys.argv[3]
#zone_folder_path = sys.argv[4]

# Create folder if it doesn't exist
if not os.path.exists(zone_folder_path):
    os.makedirs(zone_folder_path)

# Download the file
downloaded_file_path = download_file(power_plants_raw_data_download_link, power_plants_raw_data_folder_path, power_plants_raw_data_file_name)

print(f"File '{power_plants_raw_data_file_name}' downloaded and saved at: {downloaded_file_path}")
power_plants_raw_data_file_path = downloaded_file_path
print(f'power_plants_raw_data_file_path: {power_plants_raw_data_file_path}')

File '2020-01.csv' downloaded and saved at: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [6]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv


- Saving power plants raw data link sources

  The next two cells create a .csv file where all the download links used in each download action are written and so to provide a only file where all online sources can be found

In [7]:
power_plants_raw_data_download_link_sources_folder_path = zone_folder_path

In [8]:
import pandas as pd
import os

def create_csv(power_plants_raw_data_download_link_sources_file_name, power_plants_raw_data_download_link_sources_folder_path, sources_file_headers):
    # Define file path
    file_path = f"{power_plants_raw_data_download_link_sources_folder_path}/{power_plants_raw_data_download_link_sources_file_name}"
    
    # Check if the file already exists
    if os.path.exists(file_path):
        print(f"A file with the name '{power_plants_raw_data_download_link_sources_file_name}' already exists in the specified directory. CSV file not created.")
        return
    
    # Create DataFrame with empty values based on headers
    df = pd.DataFrame(columns=sources_file_headers)
    
    # Write DataFrame to CSV
    df.to_csv(file_path, index=False)
    print(f"CSV file '{power_plants_raw_data_download_link_sources_file_name}' created successfully at '{power_plants_raw_data_download_link_sources_folder_path}'.")

# Example usage
power_plants_raw_data_download_link_sources_file_name = "power_plants_raw_data_download_link_sources.csv"  # Fixed file name
#folder_path = input("Enter the folder path: ")
sources_file_headers = ["zone_folder_name", "zone_folder_path", "file_name", "download_link"]

create_csv(power_plants_raw_data_download_link_sources_file_name, power_plants_raw_data_download_link_sources_folder_path, sources_file_headers)

print(f'power_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}')
print(f'power_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}')

A file with the name 'power_plants_raw_data_download_link_sources.csv' already exists in the specified directory. CSV file not created.
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [9]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants


In [10]:
import pandas as pd
import os

def write_to_csv(file_path, data_dict):
    # Read existing CSV file
    df = pd.read_csv(file_path)
    
    # Create a DataFrame for the new row with empty values
    new_row = pd.DataFrame([data_dict])
    
    # Append the new row to the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    
    # Write the updated DataFrame back to CSV
    df.to_csv(file_path, index=False)
    print("Data written to CSV successfully.")

if __name__ == "__main__":
    # Input variables
    #power_plants_raw_data_download_link_sources_file_name = "power_plants_raw_data_download_link_sources.csv"
    #power_plants_raw_data_download_link_sources_folder_path = input("power_plants_raw_data_download_link_sources_folder_path")

    # Construct the full file path
    file_path = os.path.join(power_plants_raw_data_download_link_sources_folder_path, power_plants_raw_data_download_link_sources_file_name)
    
    # Read the existing CSV file to understand its structure
    df = pd.read_csv(file_path)
    print("Existing columns in the CSV file:")
    print(df.columns.tolist())

    # Additional data to be written
    data_dict = {
        "zone_folder_name": zone_folder_name,
        "zone_folder_path": zone_folder_path,
        "file_name": power_plants_raw_data_file_name,
        "download_link": power_plants_raw_data_download_link
    }

    # Write data to the next empty row in the CSV file
    write_to_csv(file_path, data_dict)

Existing columns in the CSV file:
['zone_folder_name', 'zone_folder_path', 'file_name', 'download_link']
Data written to CSV successfully.


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [11]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants


            3. POWER PLANTS DATA FORMATING

3.1. Defining variable as the file name of all clean data and the directory where it will be stored. 
    
    It is recommended to give the year which the data is refering to as the name of the power plants clean data file. 
    Aditionally, take account to give the appropiate file extension to the downloaded file name, .csv in this case:

    power_plants_clean_data_file_name = "20##.csv"
        e.g.
    power_plants_clean_data_file_name = "2020.csv"

In [12]:
# Define the power plants data file name
#power_plants_raw_data_folder_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/0"
power_plants_clean_data_file_name = "2020.csv"  # Replace with the desired file name
# Call the external Python script with defined variables
#%run -i Power_Plants_Data_Formating.py {power_plants_raw_data_folder_path} {power_plants_data_file_name} 

3.2. Creating .csv clean data file with its respective headers.

In [13]:
import sys
import pandas as pd
import os

def create_csv_file(folder_path, file_name, data=None):
  """
  Creates a CSV file with predefined headers using pandas.

  Args:
      folder_path (str): The path of the folder where the CSV file will be created.
      file_name (str): The name of the CSV file.
      data (list of lists or pandas.DataFrame, optional): Data to write to the CSV (default: None).

  Returns:
      None
  """

  headers = [
      "", "Unit", "PowerCapacity", "Nunits", "Zone", "Zone_th", "Zone_h2", "Technology", "Fuel", "Efficiency",
      "MinUpTime", "MinDownTime", "RampUpRate", "RampDownRate", "StartUpCost", "NoLoadCost_pu", "RampingCost",
      "PartLoadMin", "MinEfficiency", "StartUpTime", "CO2Intensity", "CHPType", "CHPPowerToHeat",
      "CHPPowerLossFactor", "CHPMaxHeat", "COP", "Tnominal", "coef_COP_a", "coef_COP_b", "STOCapacity",
      "STOSelfDischarge", "STOMaxChargingPower", "STOChargingEfficiency", "WaterWithdrawal", "WaterConsumption", "Status", "Source"
  ]

  # Create empty DataFrame if no data provided
  if data is None:
      data = []
  elif not isinstance(data, (list, pd.DataFrame)):
      raise TypeError("Data must be a list of lists or a pandas.DataFrame")

  df = pd.DataFrame(data, columns=headers)

  # Handle potential directory error and combine path with file name
  try:
      file_path = os.path.join(folder_path, file_name)
  except IsADirectoryError:
      print(f"Error: '{folder_path}' is a directory. Please provide a file name within the directory.")
      return None

  # Save DataFrame to CSV file
  df.to_csv(file_path, index=False)  # Don't include row index in CSV

  print(f"CSV file '{file_name}' created with headers: {', '.join(headers)}")
  return file_path

#power_plants_raw_data_folder_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/0"
#power_plants_clean_data_file_name = "2020.csv"

# Input variables from command line arguments
#power_plants_raw_data_folder_path = sys.argv[1]
#power_plants_data_file_name = sys.argv[2]


create_csv_file(power_plants_raw_data_folder_path, power_plants_clean_data_file_name, data=None)
file_path = create_csv_file(power_plants_raw_data_folder_path, power_plants_clean_data_file_name, data=None)

power_plants_clean_data_folder_path = power_plants_raw_data_folder_path  # Copy the path to a new variable
power_plants_clean_data_file_path = file_path

print("power_plants_clean_data_folder_path:", power_plants_clean_data_folder_path)
print("power_plants_clean_data_file_path:", power_plants_clean_data_file_path)

CSV file '2020.csv' created with headers: , Unit, PowerCapacity, Nunits, Zone, Zone_th, Zone_h2, Technology, Fuel, Efficiency, MinUpTime, MinDownTime, RampUpRate, RampDownRate, StartUpCost, NoLoadCost_pu, RampingCost, PartLoadMin, MinEfficiency, StartUpTime, CO2Intensity, CHPType, CHPPowerToHeat, CHPPowerLossFactor, CHPMaxHeat, COP, Tnominal, coef_COP_a, coef_COP_b, STOCapacity, STOSelfDischarge, STOMaxChargingPower, STOChargingEfficiency, WaterWithdrawal, WaterConsumption, Status, Source
CSV file '2020.csv' created with headers: , Unit, PowerCapacity, Nunits, Zone, Zone_th, Zone_h2, Technology, Fuel, Efficiency, MinUpTime, MinDownTime, RampUpRate, RampDownRate, StartUpCost, NoLoadCost_pu, RampingCost, PartLoadMin, MinEfficiency, StartUpTime, CO2Intensity, CHPType, CHPPowerToHeat, CHPPowerLossFactor, CHPMaxHeat, COP, Tnominal, coef_COP_a, coef_COP_b, STOCapacity, STOSelfDischarge, STOMaxChargingPower, STOChargingEfficiency, WaterWithdrawal, WaterConsumption, Status, Source
power_plants

- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [14]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path {power_plants_clean_data_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv


3.3. Copying the corresponding data to the power_plants_clean_data_file 

    This part compares headers of both files power_plants_raw_data_file and power_plants_clean_data_file to select the common ones according to the equivalencies of the power_plants_clean_data_equivalent_headers_file to finally copy all the data of the selected columns.

In [15]:
power_plants_clean_data_equivalent_headers_file_name = 'power_plants_clean_data_equivalent_headers.csv'
power_plants_clean_data_equivalent_headers_file_path = '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_clean_data_equivalent_headers.csv'

In [16]:
import pandas as pd

# Read the headers from the raw data file
raw_data_headers = pd.read_csv(power_plants_raw_data_file_path, nrows=0).columns.tolist()

# Read the equivalent headers file
equivalent_headers_df = pd.read_csv(power_plants_clean_data_equivalent_headers_file_path)

# Initialize an empty dictionary to store mapping of equivalent headers to dispaset headers
equivalent_to_dispaset_mapping = dict(zip(equivalent_headers_df['Equivalent_Headers'], equivalent_headers_df['Dispaset_Headers']))

# Read the clean data file to get all existing headers
existing_headers_df = pd.read_csv(power_plants_clean_data_file_path, nrows=0)
existing_headers = existing_headers_df.columns.tolist()

# Read the raw data file
raw_data_df = pd.read_csv(power_plants_raw_data_file_path)

# Initialize a new DataFrame to store the copied columns along with existing headers
clean_data_df = pd.DataFrame()

# Iterate over the existing headers in the clean data file
for existing_header in existing_headers:
    # Check if the existing header exists in the equivalent headers mapping
    if existing_header in equivalent_to_dispaset_mapping.values():
        # Find the corresponding equivalent header
        equivalent_header = next((key for key, value in equivalent_to_dispaset_mapping.items() if value == existing_header), None)
        if equivalent_header is not None and equivalent_header in raw_data_df.columns:
            # Copy the corresponding column from the raw data file to the clean data DataFrame
            clean_data_df[existing_header] = raw_data_df[equivalent_header]
        else:
            # Handle case where equivalent header is not found or is NaN
            clean_data_df[existing_header] = None
    else:
        # If the existing header doesn't have a corresponding equivalent, copy it as is
        clean_data_df[existing_header] = None

# Write the clean data DataFrame to the clean data file
clean_data_df.to_csv(power_plants_clean_data_file_path, index=False)

print("Columns copied successfully.")

Columns copied successfully.


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [17]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
power

3.4. Zone Filter

    This part erase all data that does not belongs to the zone/country/region/state/etc. to be modeled.
    This stept can be jumped if it would rather to use an a single file where all units of all the zones are placed in however it is recommended to keep the dispa-SET directories structure i.e. storage all the power plants data related to a zone in a separated folder.
    Additioanlly, if the name of the zone/country/region to be filtered is not the same of the zone_folder_name variable uncommen the next line and comment and enter the correponding name of the zone wanted to be filtered next comment the subsecuent line.

In [18]:
#zone_name= "Set the name of zone to be filtered"
zone_name = zone_folder_name

In [19]:
import pandas as pd

# Variables
#power_plants_clean_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv"
#zone_name = zone_folder_name

# Read CSV file
df = pd.read_csv(power_plants_clean_data_file_path)

# Filter rows where the "Zone" column matches zone_folder_name
filtered_df = df[df["Zone"] == zone_name]

# Overwrite the CSV file with the filtered DataFrame
filtered_df.to_csv(power_plants_clean_data_file_path, index=False)

print("Rows filtered successfully.")

Rows filtered successfully.


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [20]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
power

3.5.  Nunits Filter

    This part assign a number of 1 to the column called Units of the respective cell does not have any data.

In [21]:
import pandas as pd

# Variables
#power_plants_clean_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv"
Header_3 = "Nunits"

# Read CSV file
df = pd.read_csv(power_plants_clean_data_file_path)

# Fill empty cells in the "Nunits" column with 1
df[Header_3] = df[Header_3].fillna(1)

# Write the updated DataFrame back to the CSV file
df.to_csv(power_plants_clean_data_file_path, index=False)

print("Empty cells in the 'Nunits' column have been filled with 1.")

Empty cells in the 'Nunits' column have been filled with 1.


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [22]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
power

3.6.  Unit Filter

    This part joint all the units with the same name adding their respective capacity and number of units.

In [23]:
import pandas as pd

# Variables
#power_plants_clean_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv"
Header_1 = "Unit"
Header_2 = "PowerCapacity"
Header_3 = "Nunits"

# Read CSV file
df = pd.read_csv(power_plants_clean_data_file_path)

# Find duplicated cells in the "Unit" column
duplicated_units = df[df.duplicated(subset=[Header_1], keep=False)]

# Process duplicated units
for unit in duplicated_units[Header_1].unique():
    # Filter rows with the current duplicated unit
    unit_rows = df[df[Header_1] == unit]
    # Calculate the sum of "Nunits" and "PowerCapacity"
    total_nunits = unit_rows[Header_3].sum()
    total_power_capacity = unit_rows[Header_2].sum()
    # Put the sum results in the first duplicated row
    first_row_index = unit_rows.index[0]
    df.at[first_row_index, Header_3] = total_nunits
    df.at[first_row_index, Header_2] = total_power_capacity
    # Drop other duplicated rows
    df.drop(unit_rows.index[1:], inplace=True)

# Write the updated DataFrame back to the CSV file
df.to_csv(power_plants_clean_data_file_path, index=False)

print("Duplicated cells in the 'Unit' column processed successfully.")

Duplicated cells in the 'Unit' column processed successfully.


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [24]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
power

3.7 Technology Filter

    This part read the current value of the column called "technology" of power_plants_clean_data_file and makes a compariton with the contend of the power_plants_all_data_equivalent_Technologies_file and replace all the values of the column by their respective equivalent format needed to dispa-SET.
    Additionally it copies to the power_plants_all_data_not_defined_units_file all the rows of the units with not defined technology or other kind of information in that field.

In [25]:
power_plants_all_data_equivalent_technologies_file_name = "power_plants_all_data_equivalent_technologies.csv"
power_plants_all_data_equivalent_technologies_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_equivalent_technologies.csv"
power_plants_all_data_not_defined_units_file_name = "power_plants_all_data_not_defined_units.csv"
power_plants_all_data_not_defined_units_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_not_defined_units.csv"

In [26]:
print(f"power_plants_all_data_equivalent_technologies_file_name: {power_plants_all_data_equivalent_technologies_file_name}")
print(f"power_plants_all_data_equivalent_technologies_file_path: {power_plants_all_data_equivalent_technologies_file_path}")
print(f"power_plants_all_data_not_defined_units_file_name: {power_plants_all_data_not_defined_units_file_name}")
print(f"power_plants_all_data_not_defined_units_file_path: {power_plants_all_data_not_defined_units_file_path}")

power_plants_all_data_equivalent_technologies_file_name: power_plants_all_data_equivalent_technologies.csv
power_plants_all_data_equivalent_technologies_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_equivalent_technologies.csv
power_plants_all_data_not_defined_units_file_name: power_plants_all_data_not_defined_units.csv
power_plants_all_data_not_defined_units_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_not_defined_units.csv


In [27]:
import pandas as pd

# Define file paths
#power_plants_clean_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv"
#power_plants_all_data_equivalent_technologies_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_equivalent_technologies.csv"
#power_plants_all_data_not_defined_units_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_not_defined_units.csv"

# Read CSV files
clean_data_df = pd.read_csv(power_plants_clean_data_file_path)
equivalent_technologies_df = pd.read_csv(power_plants_all_data_equivalent_technologies_file_path)

# Create a dictionary mapping Equivalent_Technology to Dispaset_Technology
equivalent_technologies_dict = dict(zip(equivalent_technologies_df['Equivalent_Technology'], equivalent_technologies_df['Dispaset_Technology']))

# Create a list to store rows not found in the second file
not_defined_units = []

# Iterate over each value in the 'technology' column of the clean data file
for index, technology in enumerate(clean_data_df['Technology']):
    # Check if the technology exists in the equivalent technologies dictionary
    if technology in equivalent_technologies_dict:
        # If found, replace the value in the 'technology' column with the corresponding value from the dictionary
        clean_data_df.at[index, 'Technology'] = equivalent_technologies_dict[technology]
    else:
        # If not found, append the entire row to the list of not defined units
        not_defined_units.append(clean_data_df.iloc[index])
        
# Save not defined units (only if there are any)
if not_defined_units:
    # Create DataFrame and save only if there are rows
    not_defined_df = pd.DataFrame(not_defined_units)
    not_defined_df.to_csv(power_plants_all_data_not_defined_units_file_path, mode='a', header=False, index=False)
    print("Rows appended to", power_plants_all_data_not_defined_units_file_path)
else:
    print("No rows to save. Skipping writing to", power_plants_all_data_not_defined_units_file_path)

# Remove the rows not found in the second file from the clean data file
clean_data_df = clean_data_df[~clean_data_df['Technology'].isin(not_defined_df['Technology'])]

# Save the updated clean data file
clean_data_df.to_csv(power_plants_clean_data_file_path, index=False)


Rows appended to /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_not_defined_units.csv


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [28]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}\npower_plants_all_data_equivalent_technologies_file_name: {power_plants_all_data_equivalent_technologies_file_name}\npower_plants_all_data_equivalent_technologies_file_path: {power_plants_all_data_equivalent_technologies_file_path}\npower_plants_all_data_not_defined_units_file_name: {power_plants_all_data_not_defined_units_file_name}\mpower_plants_all_data_not_defined_units_file_path: {power_plants_all_data_not_defined_units_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
power

3.8. Fuel Filter

    This part read the current value of the column called "fuel" of power_plants_clean_data_file and makes a compariton with the contend of the power_plants_all_data_equivalent_fuels_file and replace all the values of the column by their respective equivalent format needed to dispa-SET.
    Additionally it copies to the power_plants_all_data_not_defined_units_file all the rows of the units with not defined fuel or other kind of information in that field.

In [29]:
power_plants_all_data_equivalent_fuels_file_name = "power_plants_all_data_equivalent_fuels.csv"
power_plants_all_data_equivalent_fuels_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_equivalent_fuels.csv"

In [30]:
import pandas as pd

# Define file paths
#power_plants_clean_data_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv"
#power_plants_all_data_equivalent_technologies_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_equivalent_technologies.csv"
#power_plants_all_data_not_defined_units_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_not_defined_units.csv"

# Read CSV files
clean_data_df = pd.read_csv(power_plants_clean_data_file_path)
equivalent_fuels_df = pd.read_csv(power_plants_all_data_equivalent_fuels_file_path)

# Create a dictionary mapping Equivalent_Fuels to Dispaset_Technology
equivalent_fuels_dict = dict(zip(equivalent_fuels_df['Equivalent_Fuel'], equivalent_fuels_df['Dispaset_Fuel']))

# Create a list to store rows not found in the second file
not_defined_units = []

# Iterate over each value in the 'Fuel' column of the clean data file
for index, fuel in enumerate(clean_data_df['Fuel']):
    # Check if the technology exists in the equivalent technologies dictionary
    if fuel in equivalent_fuels_dict:
        # If found, replace the value in the 'technology' column with the corresponding value from the dictionary
        clean_data_df.at[index, 'Fuel'] = equivalent_fuels_dict[fuel]
    else:
        # If not found, append the entire row to the list of not defined units
        not_defined_units.append(clean_data_df.iloc[index])

# Save not defined units (only if there are any)
if not_defined_units:
    # Create DataFrame and save only if there are rows
    not_defined_df = pd.DataFrame(not_defined_units)
    not_defined_df.to_csv(power_plants_all_data_not_defined_units_file_path, mode='a', header=False, index=False)
    print("Rows appended to", power_plants_all_data_not_defined_units_file_path)
else:
    print("No rows to save. Skipping writing to", power_plants_all_data_not_defined_units_file_path)

# Remove the rows not found in the second file from the clean data file
clean_data_df = clean_data_df[~clean_data_df['Fuel'].isin(not_defined_df['Fuel'])]

# Save the updated clean data file
clean_data_df.to_csv(power_plants_clean_data_file_path, index=False)

No rows to save. Skipping writing to /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_not_defined_units.csv


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [31]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}\npower_plants_all_data_equivalent_technologies_file_name: {power_plants_all_data_equivalent_technologies_file_name}\npower_plants_all_data_equivalent_technologies_file_path: {power_plants_all_data_equivalent_technologies_file_path}\npower_plants_all_data_not_defined_units_file_name: {power_plants_all_data_not_defined_units_file_name}\mpower_plants_all_data_not_defined_units_file_path: {power_plants_all_data_not_defined_units_file_path}\npower_plants_all_data_equivalent_fuels_file_name {power_plants_all_data_equivalent_fuels_file_name}\npower_plants_all_data_equivalent_fuels_file_path {power_plants_all_data_equivalent_fuels_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
power

3.9. CHPType Filter

    This part read the current value of the column called "CHPType" of power_plants_clean_data_file and makes a compariton with the contend of the power_plants_all_data_equivalent_CHPTypes_file and replace all the values of the column by their respective equivalent format needed to dispa-SET.
    Additionally it copies to the power_plants_all_data_not_defined_units_file all the rows of the units with not defined fuel or other kind of information in that field.

In [32]:
power_plants_all_data_equivalent_CHPTypes_file_name = "power_plants_all_data_equivalent_CHPTypes.csv"
power_plants_all_data_equivalent_CHPTypes_file_path = "/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_equivalent_CHPTypes.csv"

In [33]:
import pandas as pd

# Read CSV files
clean_data_df = pd.read_csv(power_plants_clean_data_file_path)
equivalent_CHPTypes_df = pd.read_csv(power_plants_all_data_equivalent_CHPTypes_file_path)

# Create a dictionary mapping Equivalent_Fuels to Dispaset_Technology
equivalent_CHPTypes_dict = dict(zip(equivalent_CHPTypes_df['Equivalent_CHPType'], equivalent_CHPTypes_df['Dispaset_CHPType']))

# Create a list to store rows not found in the second file or with "yes" value
not_defined_units = []

# Iterate over each value in the 'CHPType' column of the clean data file
for index, CHPType in enumerate(clean_data_df['CHPType']):
  # Check if the CHPType exists in the equivalent CHPType dictionary
  if CHPType in equivalent_CHPTypes_dict:
    # Check for "yes" value before updating CHPType
    if CHPType == "yes":
      # If "yes", append the entire row to the list of not defined units
      not_defined_units.append(clean_data_df.iloc[index])
      clean_data_df.at[index, 'CHPType'] = equivalent_CHPTypes_dict[CHPType]
    else:
      clean_data_df.at[index, 'CHPType'] = equivalent_CHPTypes_dict[CHPType]
  #else:
    # Handle cases where CHPType is not found in the dictionary
    # ... (consider appending the row to not_defined_units or other actions)
    #not_defined_units.append(clean_data_df.iloc[index])

# Save not defined units (only if there are any)
if not_defined_units:
    # Create DataFrame and save only if there are rows
    not_defined_df = pd.DataFrame(not_defined_units)
    not_defined_df.to_csv(power_plants_all_data_not_defined_units_file_path, mode='a', header=False, index=False)
    print("Rows appended to", power_plants_all_data_not_defined_units_file_path)
else:
    print("No rows to save. Skipping writing to", power_plants_all_data_not_defined_units_file_path)

# Comment out or modify the following lines if not needed
# # Remove rows not found in the second file from the clean data file (optional)
# #clean_data_df = clean_data_df[~clean_data_df['Fuel'].isin(not_defined_df['Fuel'])]  # Adjust column names if needed

# Save the updated clean data file (optional)
clean_data_df.to_csv(power_plants_clean_data_file_path, index=False)

Rows appended to /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/power_plants_all_data_not_defined_units.csv


- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [34]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path: {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path: {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}\npower_plants_all_data_equivalent_technologies_file_name: {power_plants_all_data_equivalent_technologies_file_name}\npower_plants_all_data_equivalent_technologies_file_path: {power_plants_all_data_equivalent_technologies_file_path}\npower_plants_all_data_not_defined_units_file_name: {power_plants_all_data_not_defined_units_file_name}\npower_plants_all_data_not_defined_units_file_path: {power_plants_all_data_not_defined_units_file_path}\npower_plants_all_data_equivalent_fuels_file_name {power_plants_all_data_equivalent_fuels_file_name}\npower_plants_all_data_equivalent_fuels_file_path {power_plants_all_data_equivalent_fuels_file_path}\npower_plants_all_data_equivalent_CHPTypes_file_name {power_plants_all_data_equivalent_CHPTypes_file_name}\npower_plants_all_data_equivalent_CHPTypes_file_path {power_plants_all_data_equivalent_CHPTypes_file_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
pow

3.10. Power Capacity filter

    This part erase all the units with the Power Capacity field empty or zero value.

In [35]:
import pandas as pd

def copy_and_remove_rows_with_empty_or_zero(input_file_path, column_name, destination_file_path):
    try:
        # Read the CSV file
        df = pd.read_csv(input_file_path)
    except FileNotFoundError:
        print("Input file not found.")
        return

    # Filter rows with empty or zero cells in the selected column
    rows_to_copy = df[df[column_name].isna() | (df[column_name] == 0)]

    # Write filtered rows to the destination file
    with open(destination_file_path, 'a') as file:
        for index, row in rows_to_copy.iterrows():
            file.write(','.join(map(str, row.tolist())) + '\n')

    # Remove filtered rows from the source DataFrame
    df = df[~df[column_name].isna()]
    df = df[df[column_name] != 0]

    # Overwrite the source CSV file without the removed rows
    df.to_csv(input_file_path, index=False)

# Example usage:
input_file_path = power_plants_clean_data_file_path
column_name = "PowerCapacity"
destination_file_path = power_plants_all_data_not_defined_units_file_path

copy_and_remove_rows_with_empty_or_zero(input_file_path, column_name, destination_file_path)


3.11. Effieciency Field

    This part copy the closer value of the Efficiency technical feauture from the already agregated data base using the notebook: EU_Power_Plant_Technical_Data_Base_Gathering.ipynb sellecting in base on the Technology, the fuel and the PowerCapacity features

In [36]:
EU_Power_Units_Technical_Features_File_name = 'EU_Power_Units_Technical_Features.csv'
EU_Power_Units_Technical_Features_File_path = '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/EU_Power_Units_Technical_Features/EU_Power_Units_Technical_Features.csv'

In [37]:
import pandas as pd

def copy_efficiency_values(first_file_path, second_file_path, common_columns):
    # Read CSV files into DataFrames
    first_df = pd.read_csv(first_file_path)
    second_df = pd.read_csv(second_file_path)

    # Iterate over unique values of "Technology" in the second file
    for technology in second_df['Technology'].unique():
        # Filter rows in both files where "Technology" matches
        first_filtered = first_df[first_df['Technology'] == technology]
        second_filtered = second_df[second_df['Technology'] == technology]

        # Iterate over unique values of "Fuel" in the second file
        for fuel in second_filtered['Fuel'].unique():
            # Filter rows with matching "Fuel" in both files
            first_fuel_filtered = first_filtered[first_filtered['Fuel'] == fuel]
            second_fuel_filtered = second_filtered[second_filtered['Fuel'] == fuel]

            # Iterate over rows in the second file
            for index, second_row in second_fuel_filtered.iterrows():
                # Find the row in the first file with the closest "PowerCapacity" value
                closest_row = first_fuel_filtered.iloc[(first_fuel_filtered['PowerCapacity'] - second_row['PowerCapacity']).abs().argsort()[:1]]

                # Copy "Efficiency" value from the first file to the second file
                if not closest_row.empty:
                    second_df.at[index, 'Efficiency'] = closest_row['Efficiency'].values[0]

    # Write updated DataFrame to the second file
    second_df.to_csv(second_file_path, index=False)

# Example usage:
Power_Units_Technical_Features_File_path = '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/Dictionaries/EU_Power_Units_Technical_Features/EU_Power_Units_Technical_Features.csv'
power_plants_clean_data_file_path = '/home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv'
common_columns = ['PowerCapacity', 'Technology', 'Fuel', 'Efficiency']

copy_efficiency_values(Power_Units_Technical_Features_File_path, power_plants_clean_data_file_path, common_columns)


    Setting with the value of 1 to all those units which no Efficiecy data was found.    

In [39]:
import pandas as pd

def fill_empty_efficiency_with_one(file_path):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("File not found.")
        return

    # Replace empty cells in the "Efficiency" column with 1
    df['Efficiency'].fillna(1, inplace=True)

    # Write the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)

# Example usage:
fill_empty_efficiency_with_one(power_plants_clean_data_file_path)

- verifiyng variables

  This cells are just to confirm all the file names, file paths and other information related to the data being processed.
  Also are used to ensure the inputs for next cells in order to avoid to re-enter the same information each time.

In [40]:
print (f"zone_folder_name: {zone_folder_name}\nzone_folder_path: {zone_folder_path}\npower_plants_raw_data_folder_path: {power_plants_raw_data_folder_path}\npower_plants_raw_data_file_name: {power_plants_raw_data_file_name}\npower_plants_raw_data_download_link: {power_plants_raw_data_download_link}\npower_plants_raw_data_file_path: {power_plants_raw_data_file_path}\npower_plants_raw_data_download_link_sources_file_name: {power_plants_raw_data_download_link_sources_file_name}\npower_plants_raw_data_download_link_sources_folder_path: {power_plants_raw_data_download_link_sources_folder_path}\npower_plants_clean_data_file_name: {power_plants_clean_data_file_name}\npower_plants_clean_data_folder_path: {power_plants_clean_data_folder_path}\npower_plants_clean_data_file_path: {power_plants_clean_data_file_path}\npower_plants_clean_data_equivalent_headers_file_name: {power_plants_clean_data_equivalent_headers_file_name}\npower_plants_clean_data_equivalent_headers_file_path: {power_plants_clean_data_equivalent_headers_file_path}\npower_plants_all_data_equivalent_technologies_file_name: {power_plants_all_data_equivalent_technologies_file_name}\npower_plants_all_data_equivalent_technologies_file_path: {power_plants_all_data_equivalent_technologies_file_path}\npower_plants_all_data_not_defined_units_file_name: {power_plants_all_data_not_defined_units_file_name}\npower_plants_all_data_not_defined_units_file_path: {power_plants_all_data_not_defined_units_file_path}\npower_plants_all_data_equivalent_fuels_file_name: {power_plants_all_data_equivalent_fuels_file_name}\npower_plants_all_data_equivalent_fuels_file_path: {power_plants_all_data_equivalent_fuels_file_path}\npower_plants_all_data_equivalent_CHPTypes_file_name: {power_plants_all_data_equivalent_CHPTypes_file_name}\npower_plants_all_data_equivalent_CHPTypes_file_path: {power_plants_all_data_equivalent_CHPTypes_file_path}\nEU_Power_Units_Technical_Features_File_name: {EU_Power_Units_Technical_Features_File_name}\nEU_Power_Units_Technical_Features_File_path: {EU_Power_Units_Technical_Features_File_path}")

zone_folder_name: DE
zone_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_raw_data_file_name: 2020-01.csv
power_plants_raw_data_download_link: https://data.open-power-system-data.org/conventional_power_plants/2020-10-01/conventional_power_plants_EU.csv
power_plants_raw_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020-01.csv
power_plants_raw_data_download_link_sources_file_name: power_plants_raw_data_download_link_sources.csv
power_plants_raw_data_download_link_sources_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants
power_plants_clean_data_file_name: 2020.csv
power_plants_clean_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE
power_plants_clean_data_file_path: /home/ray/Dispa-SET_Unleash/RawData/PowerPlants/DE/2020.csv
power_plants_clean_data_equivalent_headers_file_name: power_plants_clean_data_equivalent_headers.csv
pow

3.12. MinUpTime Field

3.13. MinDownTime RampUpRate

3.14. RampUpRate Field

3.15. RampDownRate Field

3.16. StartUpCost Field

3.17. NoLoadCost_pu Field

3.18. RampingCost Field

3.19. PartLoadMin Field

3.20. MinEfficiency Field

3.14. StartUpTime Field

3.14. CO2Intensity Field

3.14. RampUpRate Field

3.14. RampUpRate Field